In [1]:
import numpy as np

In [2]:
class MMIT2:
    def __init__(self, max_depth=3, margin_length=0.1, p=2, min_split_sample=2):
        # Initialize hyperparameters
        self.max_depth = max_depth
        self.margin_length = margin_length
        self.p = p
        self.min_split_sample = min_split_sample
        self.tree = None

    def relu(self, x):
        # Efficient ReLU function
        return np.maximum(0, x)

    def hinge_error(self, y_pred, y_low, y_up):
        # Vectorized hinge error calculation
        return (self.relu(y_low - y_pred + self.margin_length) ** self.p + 
                self.relu(y_pred - y_up + self.margin_length) ** self.p)

    def best_split(self, X, y):
        _, n_features = X.shape
        best_feature, best_threshold = None, None
        min_error = float('inf')

        # Precompute y_low and y_up for all samples
        y_low = y[:, 0] + self.margin_length
        y_up = y[:, 1] - self.margin_length

        for feature in range(n_features):
            # Unique thresholds for the current feature
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                # Compute masks for splitting
                left_mask = X[:, feature] <= threshold
                right_mask = ~left_mask  # Avoid recomputing

                # Ensure valid splits
                if np.sum(left_mask) < self.min_split_sample or np.sum(right_mask) < self.min_split_sample:
                    continue

                # Evaluate split hinge error
                error = (self.evaluate_split(y_low[left_mask], y_up[left_mask]) + self.evaluate_split(y_low[right_mask], y_up[right_mask]))

                # Update the best split if current split is better
                if error < min_error:
                    min_error = error
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold

    def evaluate_split(self, y_low, y_up):
        # Efficiently evaluate the hinge error for a split
        valid_values = np.concatenate([y_low[y_low != -np.inf], y_up[y_up != np.inf]])
        if valid_values.size == 0:
            return 0

        # Vectorized computation of errors for all valid values
        errors = np.sum(self.hinge_error(valid_values[:, None], y_low, y_up), axis=1)
        return np.min(errors)

    def predict_leaf_value(self, y):
        # Predict the optimal value at a leaf node
        y_low = y[:, 0] + self.margin_length
        y_up = y[:, 1] - self.margin_length
        valid_values = np.concatenate([y_low[y_low != -np.inf], y_up[y_up != np.inf]])
        if valid_values.size == 0:
            return 0

        # Vectorized computation of errors for all valid values
        errors = np.sum(self.hinge_error(valid_values[:, None], y_low, y_up), axis=1)
        return valid_values[np.argmin(errors)]

    def build_tree(self, X, y, depth):
        # Build the decision tree recursively
        if depth == self.max_depth or len(X) < self.min_split_sample:
            return {'type': 'leaf', 'value': self.predict_leaf_value(y)}

        # Find the best split
        feature, threshold = self.best_split(X, y)
        if feature is None:
            return {'type': 'leaf', 'value': self.predict_leaf_value(y)}

        # Split the data
        left_mask = X[:, feature] <= threshold
        right_mask = ~left_mask

        # Build subtrees recursively
        return {
            'type': 'node',
            'feature': feature,
            'threshold': threshold,
            'left': self.build_tree(X[left_mask], y[left_mask], depth + 1),
            'right': self.build_tree(X[right_mask], y[right_mask], depth + 1)
        }

    def fit(self, X, y):
        # Fit the decision tree to the data
        self.tree = self.build_tree(X, y, 0)

    def predict_one(self, x, node):
        # Predict the target for a single instance
        while node['type'] != 'leaf':
            if x[node['feature']] <= node['threshold']:
                node = node['left']
            else:
                node = node['right']
        return node['value']

    def predict(self, X):
        # Predict the target for all instances
        return np.array([self.predict_one(x, self.tree) for x in X])

In [3]:
import numpy as np
from joblib import Parallel, delayed

class MMIT3:
    def __init__(self, max_depth=3, margin_length=0.1, p=2, min_split_sample=2, n_jobs=-1):
        # Initialize hyperparameters
        self.max_depth = max_depth
        self.margin_length = margin_length
        self.p = p
        self.min_split_sample = min_split_sample
        self.tree = None
        self.n_jobs = n_jobs  # Number of parallel jobs (-1 means all available cores)

    def relu(self, x):
        # Efficient ReLU function
        return np.maximum(0, x)

    def hinge_error(self, y_pred, y_low, y_up):
        # Vectorized hinge error calculation
        return (self.relu(y_low - y_pred + self.margin_length) ** self.p + 
                self.relu(y_pred - y_up + self.margin_length) ** self.p)

    def evaluate_split(self, y_low, y_up):
        # Efficiently evaluate the hinge error for a split
        valid_values = np.concatenate([y_low[y_low != -np.inf], y_up[y_up != np.inf]])
        if valid_values.size == 0:
            return 0

        # Vectorized computation of errors for all valid values
        errors = np.sum(self.hinge_error(valid_values[:, None], y_low, y_up), axis=1)
        return np.min(errors)

    def evaluate_threshold(self, X, y_low, y_up, feature, threshold):
        # Compute masks for splitting
        left_mask = X[:, feature] <= threshold
        right_mask = ~left_mask  # Avoid recomputing

        # Ensure valid splits
        if np.sum(left_mask) < self.min_split_sample or np.sum(right_mask) < self.min_split_sample:
            return float('inf')

        # Evaluate split hinge error
        left_y_low, left_y_up = y_low[left_mask], y_up[left_mask]
        right_y_low, right_y_up = y_low[right_mask], y_up[right_mask]

        # Compute error for left and right branches
        return (self.evaluate_split(left_y_low, left_y_up) + 
                self.evaluate_split(right_y_low, right_y_up))

    def best_split(self, X, y):
        _, n_features = X.shape
        best_feature, best_threshold = None, None
        min_error = float('inf')

        # Precompute y_low and y_up for all samples
        y_low = y[:, 0] + self.margin_length
        y_up = y[:, 1] - self.margin_length

        # Evaluate each feature for possible splits
        for feature in range(n_features):
            thresholds = np.unique(X[:, feature])

            # Use parallel processing to evaluate thresholds
            errors = Parallel(n_jobs=self.n_jobs)(
                delayed(self.evaluate_threshold)(X, y_low, y_up, feature, threshold)
                for threshold in thresholds
            )

            # Find the best threshold for the current feature
            min_threshold_error = min(errors)
            if min_threshold_error < min_error:
                min_error = min_threshold_error
                best_feature = feature
                best_threshold = thresholds[errors.index(min_error)]

        return best_feature, best_threshold

    def predict_leaf_value(self, y):
        # Predict the optimal value at a leaf node
        y_low = y[:, 0] + self.margin_length
        y_up = y[:, 1] - self.margin_length
        valid_values = np.concatenate([y_low[y_low != -np.inf], y_up[y_up != np.inf]])
        if valid_values.size == 0:
            return 0

        # Vectorized computation of errors for all valid values
        errors = np.sum(self.hinge_error(valid_values[:, None], y_low, y_up), axis=1)
        return valid_values[np.argmin(errors)]

    def build_tree(self, X, y, depth):
        # Build the decision tree recursively
        if depth == self.max_depth or len(X) < self.min_split_sample:
            return {'type': 'leaf', 'value': self.predict_leaf_value(y)}

        # Find the best split
        feature, threshold = self.best_split(X, y)
        if feature is None:
            return {'type': 'leaf', 'value': self.predict_leaf_value(y)}

        # Split the data
        left_mask = X[:, feature] <= threshold
        right_mask = ~left_mask

        # Build subtrees recursively
        return {
            'type': 'node',
            'feature': feature,
            'threshold': threshold,
            'left': self.build_tree(X[left_mask], y[left_mask], depth + 1),
            'right': self.build_tree(X[right_mask], y[right_mask], depth + 1)
        }

    def fit(self, X, y):
        # Fit the decision tree to the data
        self.tree = self.build_tree(X, y, 0)

    def predict_one(self, x, node):
        # Predict the target for a single instance
        while node['type'] != 'leaf':
            if x[node['feature']] <= node['threshold']:
                node = node['left']
            else:
                node = node['right']
        return node['value']

    def predict(self, X):
        # Predict the target for all instances
        return np.array([self.predict_one(x, self.tree) for x in X])

In [4]:
# Example Usage
n_data = 1000
X = np.array([[i, 1] for i in range(1, n_data)])
y = np.array([[i+1, i+2] for i in range(1, n_data)])

In [5]:
dtr = MMIT2(max_depth=2, margin_length=0.1, p=2, min_split_sample=2)
dtr.fit(X, y)
y_pred2 = (dtr.predict(X))

In [6]:
dtr = MMIT3(max_depth=2, margin_length=0.1, p=2, min_split_sample=2)
dtr.fit(X, y)
y_pred3 = (dtr.predict(X))

In [7]:
np.sum(y_pred3 - y_pred2)

np.float64(0.0)